note that this notebook can be viewed as a slideshow triggered by: File -> Download as -> Reveal.js slides (.html)

## PySDM tutorial: Intro
### authors: see [github.com/atmos-cloud-sim-uj](https://github.com/atmos-cloud-sim-uj/PySDM/graphs/contributors)
### copyright: Jagiellonian University
#### tutorial licence: CC-BY

### *PySDM* : particle-based (a.k.a. super-droplet) simulations of atmospheric cloud microphysics

The package features a Pythonic implementation of the Super-Droplet Method (SDM) Monte-Carlo algorithm, hence the name.

PySDM simulates the dynamics of population of particles immersed in moist air. Depending on their size, the simulated particles are referred to as:
- atmospheric aerosol (sub-micrometre particles, acting as nuclei for condensation of water vapour),
- cloud droplets (with radius in the 1 to 100 micrometre range, with negligible fall speed), 
- rain drops (larger precipitating particles formed by collisions).

### The crux of the simulations is to compute the evolution of:
- physicochemical properties of the aerosol/cloud/rain particles (size, nucleus properties, ...),
- thermo- and hydro-dynamic properties of the surrounding air (moisture, heat, momentum, ...).


### The particle-based ansatz in a nutshell:
- the particles are modelled as point-like objects called **super-droplets**   
 (with each super-droplet representing a multiplicity $n$ of its real-world counterparts with identical properties such as radius $r$, nucleus radius $r_d$, ...);

- the **ambient air** is modelled as a perfect gas mixture of dry air and water vapour   
  (characterised by tempreture $T$, total and partial pressures $p=p_d+p_v$, density $\rho=\rho_d+\rho_v$, with $d$ and $v$ denoting dry-air and water-vapour, respectively);

- there is **biderectional coupling** between the ambient air and the particles   
  (in particular: moisture present in the air drives particle growth by condensation, concurrent latent heat release alters the air temperature).

### The following model state description is embraced:
- moist air thermodynamic variables (see [Constants](constants.ipynb) for definition of $p_{1000}$, $R_d$, $c_{pd}$): 
  - dry-air density: $\rho_d$,
  - dry-air potential temperature: $\theta_d=T/\Pi=T\left(\frac{p_{1000}}{p_d}\right)^{\frac{R_d}{c_{pd}}}$, 
  - water vapour mixing ratio: $q_v=\frac{\rho_v}{\rho_d}$,
    
- attributes of $i$-th super-droplet:
  - multiplicity: $n^{[i]}$,
  - extensive attributes: particle volume $v^{[i]}$, nucleus volume, ...,
  - intensive attributes

### Particle dynamics: collisional growth

Collisional growth is commonly modelled by the **Smoluchowski's coagulation equation** [^1]:

$\begin{eqnarray}
\frac{d}{dt}{c_i} = \frac{1}{2} \sum\limits_{j=1}^{i-1} a_{j, i-j} c_j c_{i-j} - \sum\limits_{j=1}^{\infty} a_{j, i} c_j c_i
\end{eqnarray}$

where $a_{i,j}$ is a so-called kernel defining rate of collisions and $c_i$ is the number concentration (in a volume of air) of the particles with volume $i \cdot \Delta v$ where $\Delta v$ is a droplet volume increment.

Note that $c_i$ will correspond to super-droplet multiplicity $n^{[i]}$ (in a unit volume of air) if:
- $v^{[i]} = i \cdot \Delta v$,
- $v^{[i]}$ does not change in time.

In order to relax the above conditions, a congruent stochastic formulation solved with a Monte-Carlo algorithm is used in PySDM  (see [Coalescence](coalescence.ipynb)).

[^1]: formulated as continuum integro-differential equation in [Smoluchowski 1916](references.ipynb) under the assumptions of sufficiently large system and of neglected correlations between numbers of droplets of different sizes  

### Particle dynamics: diffusional growth (1/2)

Diffusional growth is modelled with the **Fick's** law of diffusion depicting the proportionality of the flux of mass of vapour per surface of droplet of radius $r$ to the gradient of vapour density (ambient $\rho_v=q_v \rho_d$ vs. particle surface $\rho_\circ$):

$\begin{eqnarray}
\frac{\dot{m}}{4 \pi r^2} =  
  D \frac{q_\text{v}\rho_d - \rho_\circ\left(r, \rho_d, \theta_d, q_v\right)}{r}
\end{eqnarray}$

The $\rho_\circ$ is the water vapour density at the droplet surface assuming equilibrium of liquid and vapour phases (see [Condensation](Condensation.ipynb)).

### Particle dynamics: diffusional growth (2/2)

Growth of droplets is coupled with the evolution of ambient air thermodynamic properties with:

  $\frac{d}{dt} \left[\begin{eqnarray} 
    v^{[i]} \\ 
    \vdots \\
    \rho_d \\
    \\
    \theta_d \\ 
    \\
    q_v
    \\
    ~
  \end{eqnarray} \right]$
  $=\left[\begin{eqnarray} 
    \dot{m}(v^{[i]}, \rho_d, \theta_d, q_v) / \rho_l\\
    \vdots \\ 
    \partial_t \rho_d \\
    \\
    -\frac{l_v(T)}{c_p(q_v)} \frac{dq_v}{dt} \frac{1}{\Pi(\rho_d, \theta_d)} + \partial_t \theta_d  \\ 
     -\frac{\rho_l}{\rho_d} \frac{1}{\Delta V} \sum\limits_i n^{[i]} \frac{dv^{[i]}}{dt} + \partial_t q_v
  \end{eqnarray}\right]$

The $\partial_t$ derivatives denote tendencies due to ambient air motion, $l_v(T)$ is the latent heat of evaporation, $c_p()$ ... $\Pi$ is the Exner function.

### Ambient air dynamics

- PDEs in the case of Eulerian environment TODO (see, e.g., [Kinematic 2D setups](kinematic_2D_setups.ipynb)):   
$\partial_t\rho_d = -\nabla\cdot(\vec{u}\rho_d)=0$   
$\partial_t q_v = - \frac{1}{\rho_d } \nabla \cdot (\vec{u} \rho_d q_v)$   
$\partial_t \theta_d = - \frac{1}{\rho_d } \nabla \cdot (\vec{u} \rho_d \theta_d)$

- ODEs in the case of Lagrangian environments (see, e.g., [Parcel setups](parcel_setups.ipynb)):    
$\partial_t \rho_d=\dot{\rho}_d = ...$   
$\partial_t q_v = 0$   
$\partial_t \theta_d = 0$

The above justifies the choice of ($\rho_d$, $\theta_d$, $q_v$) state variable triplet.